# С учетом того, что наши лучшие веса сохранены в формате pth

In [ ]:
BATCH_SIZE = 128
MAX_ROWS = 100
MAX_COL = 9
DEVICE = 'cuda:0'

In [ ]:
from torchinfo import summary 
import torch.nn as nn
import torch
summary(model=model, input_size =(BATCH_SIZE, MAX_ROWS, MAX_COL))

In [ ]:
class Net_infe(Net):#наследуемся от предыдущей модели
    def __init__(self):
        super().__init__()
        self.softmax = nn.Softmax()
    def forward(self, x):
        x = torch.where(torch.isnan(x), torch.tensor(0.0,dtype=torch.float32).to(DEVICE),x)
        x = torch.mean(x, dim = 0, keepdim = False)
        x = self.flatten(x)
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = self.l5(x)
        return self.softmax(x)
model_infe = Net_infe()

In [ ]:
model_infe.load_state_dict(torch.load('/kaggle/working/model.pth'), strict=False)
# Аккуратно!!!! Тут путь не тот - у нас в другом файле веса!!!
model_infe = model_infe.to(DEVICE)

In [ ]:
input_size = (BATCH_SIZE,MAX_ROWS,MAX_COL)
saved_onnx = 'model.onnx'
sample_input = torch.rand(input_size).to(DEVICE)
model_infe.eval()
preds = model_infe(sample_input)
preds

In [ ]:
torch.onnx.export(
    model_infe,
    sample_input,
    saved_onnx,
    verbose=False,
    input_names=['inputs'],
    output_names=['outputs'],
    opset_version=11
)

In [ ]:
import onnx
model_onnx = onnx.load(saved_onnx)
onnx.checker.check_model(model_onnx)
print(onnx.helper.printable_graph(model_onnx.graph))


In [ ]:
!pip install onnx-tf

In [ ]:
from onnx_tf.backend import prepare

tf_rep = prepare(model_onnx)

In [ ]:
pb_path = "model.pb"
tf_rep.export_graph(pb_path)

assert os.path.exists(pb_path)
print(".pb model converted successfully.")

In [ ]:
input_nodes = tf_rep.inputs
output_nodes = tf_rep.outputs
print("The names of the input nodes are: {}".format(input_nodes))
print("The names of the output nodes are: {}".format(output_nodes))

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(pb_path)
tflite_rep = converter.convert()

tflite_model_path = 'model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_rep)

In [ ]:
!zip submission.zip $tflite_model_path

In [ ]:
!pip3 install tflite_runtime

In [ ]:
import tflite_runtime.interpreter as tflite

In [ ]:
interpreter = tflite.Interpreter(tflite_model_path)
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")

list_label = train_df['sign'].unique()#У нас словарь с ключами  по-другому называется

for i in range(100):
    frames = load_relevant_data_subset(f'{train_df.iloc[i].path}')
    output = prediction_fn(inputs=frames)
    sign = np.argmax(output["outputs"])

    print(f"Predicted label: {p2s_map[sign]}, Actual Label: {train_df.iloc[i].sign}")